In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [ ]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [ ]:
pd_raw.head()

In [ ]:
time_idx=pd_raw.columns[4:]

In [ ]:
type(time_idx[0])

In [ ]:
df_plot = pd.DataFrame({
    'date':time_idx})
df_plot.head()

In [ ]:
pd_raw[pd_raw['Country/Region']=='India'].iloc[:,10::].sum(axis=0)[-4:]

In [ ]:
country_list=['India',
              'US',
              'Spain',
              'Germany',
              'United Kingdom',
              'Korea, South',
              'Japan'
             ] 

In [ ]:
for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

In [ ]:
df_plot.head()

In [ ]:
df_plot.set_index('date').plot()

In [ ]:
time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date]
time_str=[each.strftime('%Y-%m-%d') for each in time_idx] # convert back to date ISO norm (str)
print(type(time_str[0]))


In [ ]:
df_plot['date']=time_idx

type(df_plot['date'][0])
df_plot.date[1:3]

In [ ]:
df_plot.head()

In [ ]:
df_plot.to_csv('../data/processed/COVID_small_flat_table.csv',sep=';',index=False)

### Relational data model - defining a primary key


In [ ]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [ ]:
pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
pd_data_base['state']=pd_data_base['state'].fillna('no')  #ensure a sring, important for indexing

In [ ]:
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

In [ ]:
pd_relational_model=pd_data_base.set_index(['state','country'])
pd_relational_model.head()

In [ ]:
pd_relational_model=pd_relational_model.T
pd_relational_model.head()

In [ ]:
pd_relational_model=pd_relational_model.stack(level=[0,1]).reset_index()

In [ ]:
pd_relational_model = pd_relational_model.rename(columns={'level_0':'date',0:'confirmed'})
pd_relational_model.head()

In [ ]:
type(pd_relational_model['date'][0])

In [ ]:
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
pd_relational_model.confirmed=pd_relational_model.confirmed.astype(int)

pd_relational_model.dtypes

In [ ]:
pd_relational_model.head()

In [ ]:
pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

In [ ]:
test_data=pd_JH_data[((pd_JH_data['country']=='India')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [ ]:
test_data.tail()

In [ ]:
test_data.groupby(['country']).agg(np.max)

In [ ]:
test_data.groupby(['state', 'country']).agg(np.max)

In [ ]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    
    X = np.arange(-1,2).reshape(-1, 1)
    

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    print(slope,intercept)

    return intercept/slope


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

In [ ]:
%run ../src/features/build_features.py

In [ ]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [ ]:
test_data[['state','country', 'confirmed']].tail()

In [ ]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

In [ ]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

In [ ]:
pd_DR_result = pd_DR_result.reset_index()

In [ ]:
pd_DR_result

In [ ]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.tail()

In [ ]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

In [ ]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.tail()

In [ ]:
pd_result_larg[pd_result_larg['country']=='Germany'].head()

## Filtering the data with groupby apply 

In [ ]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [ ]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [ ]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

In [ ]:
pd_result_larg[pd_result_larg['country']=='Germany'].head()

In [ ]:
import plotly.graph_objects as go
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

## Filtered doubling rate

In [ ]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

In [ ]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

In [ ]:
pd_result_larg.date

In [ ]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [ ]:
pd_result_larg.head()

In [ ]:
quick_plot(pd_result_larg[pd_result_larg['country']=='Germany'].date,
           pd_result_larg[pd_result_larg['country']=='Germany'].iloc[:,[4,5,6,7]],
           y_scale='log',
           slider=True)

In [ ]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)

In [ ]:
pd_JH_data_deaths=pd.read_csv('../data/processed/COVID_relational_deaths.csv',sep=';',parse_dates=[0])
pd_JH_data_deaths=pd_JH_data_deaths.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data_deaths[pd_JH_data_deaths['country']=='India'].tail()

In [ ]:
pd_DR_result_death = pd_JH_data_deaths[['state','country','deaths']].reset_index()

In [ ]:
pd_DR_result_death.head()

In [ ]:
pd_result_larg=pd.merge(pd_result_larg,pd_DR_result_death[['index','deaths']],on=['index'],how='left')
pd_result_larg.tail()

In [ ]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)

In [ ]:
pd_JH_data_recov=pd.read_csv('../data/processed/COVID_relational_recovered.csv',sep=';',parse_dates=[0])
pd_JH_data_recov=pd_JH_data_recov.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data_recov[pd_JH_data_recov['country']=='India'].tail()

In [ ]:
pd_DR_result_recov = pd_JH_data_recov[['state','date','country','recovered']].reset_index()

In [ ]:
pd_DR_result_recov.tail(1000
)

In [ ]:
pd_DR_result_recov.to_csv('../data/processed/COVID_final_recov_set.csv',sep=';',index=False)

In [ ]:
test = pd_result_larg[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date','deaths']].groupby(['country','date']).agg(np.sum).reset_index()

In [ ]:
list(test[test['country']=='India']['confirmed'])[1:10]

In [ ]:
a = [1,2,3,4]
a[-1:]